In [1]:
import pandas as pd
import random
import numpy as np
import networkx as nx
import nltk
import tensorflow as tf
import collections
import math
import os.path
from six.moves import xrange
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import Birch
import  glob
import pickle
import anytree

In [4]:
dc = open('./dic.pkl','rb')
rdc = open('./rev_dic.pkl', 'rb')
bottom = pickle.load(dc)
top = pickle.load(rdc)
dc.close()
rdc.close()

embeds = np.load('./fembd.npy')

In [217]:
most_top = list(top.keys())[-7:]

In [74]:
def sample(label, top, parent=None):
    if label > 1999:
        node = anytree.Node(label, parent=parent)
        total = sum(list(top[label].values()))
        probabilities = [top[label][item]/total 
                                 for item in top[label]]
        left = [item[0] for item in top[label]]
        right = [item[1] for item in top[label]]
        l_choice = np.random.choice(left, p=probabilities)
        r_choice = np.random.choice(right, p=probabilities)
        sample(l_choice, top, node)
        sample(r_choice, top, node)
    else:
        parent = anytree.Node(label, parent=parent)
        node = anytree.Node(top[label], parent=parent)
    
    return node
            

In [242]:
def sample_sent(label, top):
    if label > 1999:
        total = sum(list(top[label].values()))
        probabilities = [top[label][item]/total 
                                 for item in top[label]]
        choices = list(top[label].keys())
        indices = [i for i in range(len(choices))]
        choice = choices[np.random.choice(indices, p=probabilities)]
        l_sent = sample_sent(choice[0], top)
        r_sent = sample_sent(choice[1], top)
        
        return l_sent +  r_sent
    else:
        return [label]
    


In [244]:
test = sample_sent(10000, top)

In [248]:
test.remove(0)

In [249]:
" ".join([top[label] for label in test])

'a sciences has optical fiber where species describe morphological related to being applied - UNK more reliable community , latitude UNK UNK UNK can synthesize contribute to UNK a a general'

In [224]:
samp = sample(10000, top)

for pre, _, node in anytree.RenderTree(samp):
    print("%s%s" % (pre, node.name))

10000
├── 9205
│   ├── 8798
│   │   ├── 6484
│   │   │   ├── 4414
│   │   │   │   ├── 3
│   │   │   │   │   └── .
│   │   │   │   └── 12
│   │   │   │       └── The
│   │   │   └── 4188
│   │   │       ├── 12
│   │   │       │   └── The
│   │   │       └── 170
│   │   │           └── focus
│   │   └── 6647
│   │       ├── 4521
│   │       │   ├── 865
│   │       │   │   └── discrete
│   │       │   └── 522
│   │       │       └── classes
│   │       └── 4202
│   │           ├── 157
│   │           │   └── performance
│   │           └── 2
│   │               └── of
│   └── 8516
│       ├── 7434
│       │   ├── 5879
│       │   │   ├── 1295
│       │   │   │   └── record
│       │   │   └── 0
│       │   │       └── UNK
│       │   └── 4401
│       │       ├── 0
│       │       │   └── UNK
│       │       └── 9
│       │           └── a
│       └── 6381
│           ├── 5040
│           │   ├── 1670
│           │   │   └── establishing
│           │   └── 9
│           │       └── a
│   

In [95]:
class Reason:
    def __init__(self, label, top):
        if label > 2000:
            self.l_children = [Reason(item[0], top) for item in top[label]]
            self.r_children = [Reason(item[1], top) for item in top[label]]
            total = sum(top[label][item] for item in top[label])
            self.probabilities = [top[label][item]/total for item in top[label]]
        else:
            self.children = [item for item in top[label]]
        
        self.label = label
    
    def sample(self):
        if self.label > 2000:
            return np.random.choice(self.l_children, size=1, 
                                    p=self.probabilities)(),\
                    np.random.choice(self.r_children, size=1,
                                    p=self.probabilities)()
        else:
            return top[label]

    

In [91]:
class R_Tree:
    def __init__(self, m_top, top):
        total = sum(sum(list(top[item].values())) 
                           for item in m_top)
        self.probabilities = [sum(list(top[item].values()))/total 
                             for item in m_top]
        
        self.children = [Reason(child, top) for child in m_top]
        
    def sample(self):
        return np.random.choice(self.children, size=1, 
                                p=self.probabilities)
        

In [47]:
from functools import partial

class Infix(object):
    def __init__(self, func):
        self.func = func
    def __or__(self, other):
        return self.func(other)
    def __ror__(self, other):
        return Infix(partial(self.func, other))
    def __call__(self, v1, v2):
        return self.func(v1, v2)